In [11]:
import pandas as pd

# Load the datasets (ensure these CSV files are in your notebook's working directory)
patients_df = pd.read_csv('patients_data.csv')
examinations_df = pd.read_csv('examinations_data.csv')
treatments_df = pd.read_csv('treatments_data.csv')

# Display the first few rows of each dataset for inspection
print("Patients Data:")
print(patients_df.head())

print("\nExaminations Data:")
print(examinations_df.head())

print("\nTreatments Data:")
print(treatments_df.head())


Patients Data:
  patient_id         date_time  age  gender  weight  height    BMI  \
0   9340f688  2024-04-10 20:27   51   Other    61.5   187.5  17.49   
1   87cce94b  2024-11-25 11:38   92    Male    66.5   173.2  22.17   
2   43a7c381  2024-07-03 14:18   77   Other    45.1   172.6  15.14   
3   d4a4f0b6  2024-12-07 07:24   43    Male   106.2   179.6  32.92   
4   d3e77792  2024-10-25 05:16    9  Female    80.2   155.8  33.04   

  medical_history allergies family_history  
0   Heart Disease     Latex   Hypertension  
1          Asthma   Peanuts         Cancer  
2          Asthma    Pollen       Diabetes  
3          Asthma       NaN   Hypertension  
4        Diabetes    Pollen   Hypertension  

Examinations Data:
    exam_id patient_id  heart_rate blood_pressure  oxygen_saturation  \
0  c537c658   9340f688          72     97/83 mmHg                 97   
1  0eca9a26   87cce94b          67     93/75 mmHg                 99   
2  9ff54321   43a7c381         109    111/83 mmHg         

In [12]:
def get_patient_details(patient_id):
    """
    Extract patient details from the patients_df based on patient_id.
    """
    patient_record = patients_df[patients_df['patient_id'] == patient_id]
    if patient_record.empty:
        return {}
    return patient_record.to_dict('records')[0]

def get_examination_details(patient_id):
    """
    Retrieve examination records for a given patient.
    """
    exam_records = examinations_df[examinations_df['patient_id'] == patient_id]
    return exam_records.to_dict('records')

def get_treatment_details(patient_id):
    """
    Retrieve treatment records for a given patient by linking to the examinations dataset.
    """
    # Join treatments_df with examinations_df on 'exam_id' to get 'patient_id'
    merged_df = treatments_df.merge(examinations_df[['exam_id', 'patient_id']], on='exam_id', how='left')
    
    # Filter by patient_id
    treatment_records = merged_df[merged_df['patient_id'] == patient_id]
    
    return treatment_records.to_dict('records')


In [13]:
# Select a valid patient_id from the dataset for testing
sample_patient_id = patients_df['patient_id'].iloc[0]  # Select the first patient ID

# Test patient details extraction
print("Patient Details:")
print(get_patient_details(sample_patient_id))

# Test examination details extraction
print("\nExamination Details:")
print(get_examination_details(sample_patient_id))

# Test treatment details extraction
print("\nTreatment Details:")
print(get_treatment_details(sample_patient_id))


Patient Details:
{'patient_id': '9340f688', 'date_time': '2024-04-10 20:27', 'age': 51, 'gender': 'Other', 'weight': 61.5, 'height': 187.5, 'BMI': 17.49, 'medical_history': 'Heart Disease', 'allergies': 'Latex', 'family_history': 'Hypertension'}

Examination Details:
[{'exam_id': 'c537c658', 'patient_id': '9340f688', 'heart_rate': 72, 'blood_pressure': '97/83 mmHg', 'oxygen_saturation': 97, 'temperature': 40.0, 'chief_complaint': 'Wheezing', 'symptoms': 'Wheezing, Shortness of Breath, Chest Tightness', 'pain_level': 9, 'mental_status': 'Confused', 'observations': 'Patient reports Wheezing. Symptoms include Wheezing, Shortness of Breath, Chest Tightness.', 'disease': 'Asthma', 'diagnosis': 'Asthma suspected based on symptoms and vitals.'}]

Treatment Details:
[{'treatment_id': 'd9ee3577', 'exam_id': 'c537c658', 'is_test_needed': 'Yes', 'test_name': 'Pulmonary Function Test', 'test_results': 'Pulmonary Function Test confirms Asthma.', 'medications_administered': 'Inhaler, Steroids', 'tre

In [14]:
def generate_medical_document(patient_id, nurse_notes, dentist_notes):
    """
    Generate a structured medical document combining data from CSV files and input notes.
    """
    # Retrieve information from datasets
    patient_info = get_patient_details(patient_id)
    exam_info = get_examination_details(patient_id)
    treatment_info = get_treatment_details(patient_id)
    
    # Start building the document
    document = f"---\n📝 **Medical Document for Patient ID: {patient_id}**\n---\n\n"

    # 🏥 Patient Details Section
    document += "🔹 **Patient Details:**\n"
    if patient_info:
        for key, value in patient_info.items():
            document += f"  - {key.replace('_', ' ').title()}: {value}\n"
    else:
        document += "  - No patient details found.\n"
    
    # 🔬 Examination Details Section
    document += "\n🔍 **Examination Details:**\n"
    if exam_info:
        for i, exam in enumerate(exam_info, 1):
            document += f"  ➡️ **Examination {i}:**\n"
            for key, value in exam.items():
                if key != "patient_id":  # Exclude patient_id since we already have it
                    document += f"    - {key.replace('_', ' ').title()}: {value}\n"
    else:
        document += "  - No examination records found.\n"

    # 💊 Treatment Details Section
    document += "\n💉 **Treatment Details:**\n"
    if treatment_info:
        for i, treatment in enumerate(treatment_info, 1):
            document += f"  ➡️ **Treatment {i}:**\n"
            for key, value in treatment.items():
                if key not in ["exam_id", "patient_id"]:  # Exclude linking keys
                    document += f"    - {key.replace('_', ' ').title()}: {value}\n"
    else:
        document += "  - No treatment records found.\n"

    # 🏥 Combined Observations Section
    document += "\n📝 **Observations from Medical Staff:**\n"
    document += f"  - **Nurse's Notes:** {nurse_notes}\n"
    document += f"  - **Dentist's Notes:** {dentist_notes}\n"

    return document


In [15]:
# Sample test inputs
sample_patient_id = patients_df['patient_id'].iloc[0]  # Select a valid patient ID
sample_nurse_notes = "Patient reports dizziness and mild headache."
sample_dentist_notes = "Routine dental check-up completed, no cavities found."

# Generate and print the document
generated_document = generate_medical_document(sample_patient_id, sample_nurse_notes, sample_dentist_notes)
print(generated_document)


---
📝 **Medical Document for Patient ID: 9340f688**
---

🔹 **Patient Details:**
  - Patient Id: 9340f688
  - Date Time: 2024-04-10 20:27
  - Age: 51
  - Gender: Other
  - Weight: 61.5
  - Height: 187.5
  - Bmi: 17.49
  - Medical History: Heart Disease
  - Allergies: Latex
  - Family History: Hypertension

🔍 **Examination Details:**
  ➡️ **Examination 1:**
    - Exam Id: c537c658
    - Heart Rate: 72
    - Blood Pressure: 97/83 mmHg
    - Oxygen Saturation: 97
    - Temperature: 40.0
    - Chief Complaint: Wheezing
    - Symptoms: Wheezing, Shortness of Breath, Chest Tightness
    - Pain Level: 9
    - Mental Status: Confused
    - Observations: Patient reports Wheezing. Symptoms include Wheezing, Shortness of Breath, Chest Tightness.
    - Disease: Asthma
    - Diagnosis: Asthma suspected based on symptoms and vitals.

💉 **Treatment Details:**
  ➡️ **Treatment 1:**
    - Treatment Id: d9ee3577
    - Is Test Needed: Yes
    - Test Name: Pulmonary Function Test
    - Test Results: Pulmon

In [16]:
# Test with a sample patient ID
sample_patient_id = "P001"

# Fetch and print extracted patient, examination, and treatment details
print("\nPatient Details:", get_patient_details(sample_patient_id))
print("\nExamination Details:", get_examination_details(sample_patient_id))
print("\nTreatment Details:", get_treatment_details(sample_patient_id))

# Generate the medical document and print
generated_document = generate_medical_document(sample_patient_id, "", "")
print("\nGenerated Medical Document:\n", generated_document)



Patient Details: {}

Examination Details: []

Treatment Details: []

Generated Medical Document:
 ---
📝 **Medical Document for Patient ID: P001**
---

🔹 **Patient Details:**
  - No patient details found.

🔍 **Examination Details:**
  - No examination records found.

💉 **Treatment Details:**
  - No treatment records found.

📝 **Observations from Medical Staff:**
  - **Nurse's Notes:** 
  - **Dentist's Notes:** 



In [17]:
print("Available Patient IDs in Dataset:")
print(patients_df["patient_id"].head(10))  # Print first 10 patient IDs


Available Patient IDs in Dataset:
0    9340f688
1    87cce94b
2    43a7c381
3    d4a4f0b6
4    d3e77792
5    2f812d84
6    5b053f96
7    125f58eb
8    bd868908
9    8cfe70c9
Name: patient_id, dtype: object


In [18]:
# Select a valid patient_id from the dataset
sample_patient_id = patients_df["patient_id"].iloc[0]  # Get the first valid patient ID

# Fetch and print extracted patient, examination, and treatment details
print("\nPatient Details:", get_patient_details(sample_patient_id))
print("\nExamination Details:", get_examination_details(sample_patient_id))
print("\nTreatment Details:", get_treatment_details(sample_patient_id))

# Generate the medical document and print
generated_document = generate_medical_document(sample_patient_id, "", "")
print("\nGenerated Medical Document:\n", generated_document)



Patient Details: {'patient_id': '9340f688', 'date_time': '2024-04-10 20:27', 'age': 51, 'gender': 'Other', 'weight': 61.5, 'height': 187.5, 'BMI': 17.49, 'medical_history': 'Heart Disease', 'allergies': 'Latex', 'family_history': 'Hypertension'}

Examination Details: [{'exam_id': 'c537c658', 'patient_id': '9340f688', 'heart_rate': 72, 'blood_pressure': '97/83 mmHg', 'oxygen_saturation': 97, 'temperature': 40.0, 'chief_complaint': 'Wheezing', 'symptoms': 'Wheezing, Shortness of Breath, Chest Tightness', 'pain_level': 9, 'mental_status': 'Confused', 'observations': 'Patient reports Wheezing. Symptoms include Wheezing, Shortness of Breath, Chest Tightness.', 'disease': 'Asthma', 'diagnosis': 'Asthma suspected based on symptoms and vitals.'}]

Treatment Details: [{'treatment_id': 'd9ee3577', 'exam_id': 'c537c658', 'is_test_needed': 'Yes', 'test_name': 'Pulmonary Function Test', 'test_results': 'Pulmonary Function Test confirms Asthma.', 'medications_administered': 'Inhaler, Steroids', 'tr

In [21]:
# ✅ Load Summarization Model
from transformers import pipeline

summarizer = pipeline("summarization", model="t5-small")

# ✅ Summarize Notes With Dynamic Length
def summarize_notes(doctor_notes, nurse_notes):
    combined_text = doctor_notes + " " + nurse_notes
    summary = summarizer(combined_text, max_length=len(combined_text.split()) // 2, do_sample=False)
    return summary[0]["summary_text"]


Device set to use cpu


In [22]:
# ✅ Import Necessary Libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

# ✅ Ensure 'full_text' Column Exists
if "full_text" not in examinations_df.columns:
    examinations_df["full_text"] = examinations_df["symptoms"].astype(str) + " " + examinations_df["diagnosis"].astype(str)

# ✅ Step 1: Define Custom Input
sample_nurse_notes = "Patient has been experiencing persistent cough and fever."
sample_doctor_notes = "Suspected case of respiratory infection. Prescribed antibiotics."

# ✅ Step 2: Generate AI Summary (Using LLM with Dynamic Length)
generated_summary = summarize_notes(sample_doctor_notes, sample_nurse_notes)

# ✅ Step 3: Convert Summary into Vectorized Format
vectorizer = TfidfVectorizer(stop_words="english")
X_vectorized = vectorizer.fit_transform(examinations_df["full_text"])  # Fit on dataset
query_vector = vectorizer.transform([generated_summary])  # Transform new input

# ✅ Step 4: Train a Naive Bayes Model
X_train, X_test, y_train, y_test = train_test_split(
    X_vectorized, examinations_df["diagnosis"], test_size=0.2, random_state=42, shuffle=True
)

model = MultinomialNB()
model.fit(X_train, y_train)

# ✅ Step 5: Predict Diagnosis for Custom Input
predicted_diagnosis = model.predict(query_vector)[0]

# ✅ Step 6: Compute Accuracy, Precision, and Recall
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

# ✅ Step 7: Display Results
print("🔹 AI-Generated Summary:")
print(generated_summary)

print("\n🔹 Model Evaluation Results After Custom Input:")
print(f"✅ Predicted Diagnosis: {predicted_diagnosis}")
print(f"✅ Accuracy: {accuracy:.2f}")
print(f"✅ Precision: {precision:.2f}")
print(f"✅ Recall: {recall:.2f}")


Your min_length=30 must be inferior than your max_length=7.
C:\Users\parch\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\generation\utils.py:1432: UserWarning: Unfeasible length constraints: `min_length` (30) is larger than the maximum possible length (7). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


🔹 AI-Generated Summary:
patient has been experiencing persistent cough

🔹 Model Evaluation Results After Custom Input:
✅ Predicted Diagnosis: Tuberculosis suspected based on symptoms and vitals.
✅ Accuracy: 1.00
✅ Precision: 1.00
✅ Recall: 1.00
